In [33]:
import random
import os
import ssl
import cv2
import numpy as np
import imageio
from IPython import display
from urllib import request
import re
import tempfile
import math
from tqdm.notebook import tqdm
import copy
import shutil

In [34]:
from mmflow.apis import init_model, inference_model
from mmflow.datasets import visualize_flow, write_flow
import mmcv
config_file = r"C:\Users\hikar\Documents\pwcnet_ft_4x1_300k_sintel_final_384x768.py"
checkpoint_file = r"C:\Users\hikar\Documents\pwcnet_ft_4x1_300k_sintel_final_384x768.pth"

In [35]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [36]:
# fetch videos from UCF101 dataset
UCF_ROOT = "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/"
_VIDEO_LIST = None
unverified_context = ssl._create_unverified_context()
UCF_DATA_FOLDER = r"C:\Users\hikar\Documents\UCF_101"

def list_ucf_videos():
    global _VIDEO_LIST
    if not _VIDEO_LIST:
        index = request.urlopen(UCF_ROOT, context=unverified_context).read().decode("utf-8")
        videos = re.findall("(v_[\w_]+\.avi)", index)
        _VIDEO_LIST = sorted(set(videos))
    return list(_VIDEO_LIST)
  
def path_modify(video):
  action_dir = video.split('_')[1]
  if action_dir == "HandstandPushups":
      action_dir = "HandStandPushups"
      text_list = video.split('_')
      base_video_path = text_list[0]+ '_' + action_dir + '_' + text_list[2] + '_' + text_list[3]
      print(base_video_path)
      return base_video_path
  
  return video

In [37]:
def fetch_ucf_action_list():
  cache_list = []
  video_list = list_ucf_videos()
  for path in video_list:
    action_dir = path.split('_')[1]
    if action_dir == "HandstandPushups":
      action_dir = "HandStandPushups"
    cache_list.append(action_dir)
  cache_list = set(cache_list)
  cache_list = list(cache_list)
  cache_list.sort()
  return cache_list

In [38]:
def flo_out(model,img1,img2,img3):#コードかえる
  

    
  base_path_1 = os.path.splitext(os.path.basename(img1))[0]
  base_path_2 = os.path.splitext(os.path.basename(img2))[0]
  base_path_3 = os.path.splitext(os.path.basename(img3))[0]

# D:\GR_data\capture_all_frame\Fixed\Archery\frames_v_Archery_g02_c01
#"D:\GR_data\write_flow_folder"

  flo_root_folder = r"D:\GR_data\write_flow_folder" #変更可
  split_path_list = img1.split(os.sep)
  flo_dir_name = os.path.join(flo_root_folder,split_path_list[3],split_path_list[4],split_path_list[5])#変更可
  
  if not os.path.isdir(flo_dir_name):
    os.makedirs(flo_dir_name)
#     print(flo_dir_name)

  flo_name_12 = os.path.join(flo_dir_name,base_path_1+base_path_2+".flo")
#   print(flo_name_12)
  flo_name_23 = os.path.join(flo_dir_name,base_path_2+base_path_3+".flo")
#   print(flo_name_23)

  if not os.path.exists(flo_name_12):
    result_12 = inference_model(model, img1, img2)
    write_flow(result_12,flow_file = flo_name_12)
    
  if not os.path.exists(flo_name_23):
    result_23 = inference_model(model, img2, img3)
    write_flow(result_23,flow_file = flo_name_23)

  # visualize_flow(result_12, save_file='flow_map_12.png')
  # visualize_flow(-result_12,save_file = 'flow_map_reverse.png')
  # visualize_flow(result_23,save_file = "flow_map_23.png")
  return flo_name_12,flo_name_23

In [39]:
def remove_values_closeto_0(flow_map12,flow_map23):
#   print(flow_map12)
#   print(flow_map23)
    
  condition_12_1 = (flow_map12 < 0.01)
  condition_12_2 = (flow_map12 > -0.01)
  condition_12 = condition_12_1 & condition_12_2

  condition_23_1 = (flow_map23 < 0.01)
  condition_23_2 = (flow_map23 > 0.01)
  condition_23 = condition_23_1 & condition_23_2

  flow_map12[condition_12] = 0
  flow_map23[condition_23] = 0

  return flow_map12,flow_map23


def read_2_flo(flow12_path,flow23_path):
#   print(flow23_path)
  flow_map12 = cv2.readOpticalFlow(flow12_path)
  flow_map23 = cv2.readOpticalFlow(flow23_path)

  flow_map12,flow_map23 = remove_values_closeto_0(flow_map12,flow_map23)


  return flow_map12,flow_map23

In [40]:
def convert_nan_to_0(flow_magnitude):
  nans = np.isnan(flow_magnitude)
  if np.any(nans):
      nans = np.where(nans)
      flow_magnitude[nans] = 0.
  
  return flow_magnitude


def flow_magnitude_out(flow_map12,flow_map23):
  flow_map_12_channel_0 = flow_map12[:,:,0]
  flow_map_12_channel_1 = flow_map12[:,:,1]
  
  flow_map_23_channel_0 = flow_map23[:,:,0]
  flow_map_23_channel_1 = flow_map23[:,:,1]
  
  flow_magnitude_12, _ = cv2.cartToPolar(flow_map_12_channel_0, flow_map_12_channel_1)
  flow_magnitude_23, _ = cv2.cartToPolar(flow_map_23_channel_0, flow_map_23_channel_1)
  # A couple times, we've gotten NaNs out of the above...
  
  flow_magnitude_12 = convert_nan_to_0(flow_magnitude_12)
  flow_magnitude_23 = convert_nan_to_0(flow_magnitude_23)

  return flow_magnitude_12,flow_magnitude_23

In [41]:
def warp_second_frame_to_first(flow_map_origin,second_frame):
  flow_map = copy.copy(flow_map_origin)
  h,w = flow_map.shape[:2]
  flow_map[:,:,0] += np.arange(w)#変換後の座標を指定
  flow_map[:,:,1] += np.arange(h)[:,np.newaxis]
  img2 = cv2.imread(second_frame)
  warped_prevImg = cv2.remap(img2, flow_map, None, cv2.INTER_LINEAR)#INTER_NEAREST#INTER_AREA
#   cv2.imwrite("warped_first_frame.png",warped_prevImg)

  return warped_prevImg

In [42]:
def Fixed_list_ucf(classify = "Fixed",div = 0):
    cache_list = []
    action_list = fetch_ucf_action_list()

    classify_dir = os.path.join(r"D:\GR_data\capture_crop_256",classify)#変更可
    for action in action_list:
        action_path = os.path.join(classify_dir,action)
        each_video_list = os.listdir(action_path)
        cache_list.append(each_video_list)
    return np.hstack(cache_list)

In [43]:
def frame_folder_full_path(folder_name,classify):#ex:file_name = v_ApplyEyeMakeup_g01_c01
  #ex:folder_name = frames_v_BenchPress_g24_c04
  action_dir = folder_name.split('_')[2]
  folder_name = os.path.join(r"D:\GR_data\capture_crop_256",classify,action_dir,folder_name)#変更可
  # folder_name = "/content/drive/MyDrive/UCF101_every_frame/" + classify + "/"+ action_dir + folder_name
#   print(folder_name)
  return folder_name

In [44]:
def choice_three_frame(model):#()

#Set the first 3 frames
  img1 = r"D:\GR_data\capture_crop_256\Fixed\PlayingDaf\frames_v_PlayingDaf_g02_c04\000010.png"
#   print(img1)
  img2 = r"D:\GR_data\capture_crop_256\Fixed\PlayingDaf\frames_v_PlayingDaf_g02_c04\000011.png"
#   print(img2)
  img3 = r"D:\GR_data\capture_crop_256\Fixed\PlayingDaf\frames_v_PlayingDaf_g02_c04\000012.png"
#   print(img3)


    
    
  flo1 ,flo2 = flo_out(model,img1,img2,img3)#imgからflowをoutしてそのファイルpathを返す
  flow_map12 ,flow_map23 = read_2_flo(flo1,flo2)
  flow_magnitude_12,flow_magnitude_23 = flow_magnitude_out(flow_map12,flow_map23)


  warped_prevImg2_to_1 = warp_second_frame_to_first(flow_map12,img2)
  warped_prevImg3_to_2 = warp_second_frame_to_first(flow_map23,img3)
  
  warped_1_path = os.path.join(r"C:\Users\hikar\Documents\graduation_research\warped_frame","warped_prevImg2_to_1.png")
  warped_2_path = os.path.join(r"C:\Users\hikar\Documents\graduation_research\warped_frame","warped_prevImg3_to_2.png")

  cv2.imwrite(warped_1_path,warped_prevImg2_to_1)
  cv2.imwrite(warped_2_path,warped_prevImg3_to_2)
    
  return flow_map12 ,flow_map23

In [45]:
# build the model from a config file and a checkpoint file
model = init_model(config_file, checkpoint_file, device='cuda:0')

load checkpoint from local path: C:\Users\hikar\Documents\pwcnet_ft_4x1_300k_sintel_final_384x768.pth


In [46]:
opticalflow_1,opticalflow_2 = choice_three_frame(model)

Opticalflow visualize

In [47]:
from mmflow.apis import init_model, inference_model
from mmflow.datasets import visualize_flow, write_flow
import mmcv

In [48]:
visualize_flow(opticalflow_1, save_file=r'C:\Users\hikar\Documents\graduation_research\warped_frame\flow_map_1.png')
visualize_flow(opticalflow_2, save_file=r'C:\Users\hikar\Documents\graduation_research\warped_frame\flow_map_2.png')

array([[[243, 245, 255],
        [243, 245, 255],
        [243, 245, 255],
        ...,
        [255, 253, 247],
        [255, 253, 247],
        [255, 253, 247]],

       [[243, 245, 255],
        [243, 245, 255],
        [243, 245, 255],
        ...,
        [255, 253, 247],
        [255, 253, 247],
        [255, 253, 247]],

       [[243, 245, 255],
        [243, 245, 255],
        [243, 245, 255],
        ...,
        [255, 253, 247],
        [255, 253, 247],
        [255, 253, 247]],

       ...,

       [[250, 255, 250],
        [250, 255, 250],
        [250, 255, 250],
        ...,
        [255, 251, 251],
        [255, 251, 251],
        [255, 251, 251]],

       [[250, 255, 250],
        [250, 255, 250],
        [250, 255, 250],
        ...,
        [255, 251, 250],
        [255, 251, 250],
        [255, 251, 250]],

       [[250, 255, 250],
        [250, 255, 250],
        [250, 255, 250],
        ...,
        [255, 251, 250],
        [255, 251, 250],
        [255, 251, 250]]